DATA PREPARATION - EMBEDDINGS

In [ ]:
##### Run this cell one time only
from train_test_split import train_test_set_split

FILE_PATH = r'data\sequence_donor_acceptor_100_clean.csv'
CHROM_COLUMN = 'CHROM'
TEST_CHROMS = ['chr20', 'chr21']
SPLICE_COLUMN = 'Splicing_types'

train, test = train_test_set_split(FILE_PATH, CHROM_COLUMN, TEST_CHROMS, SPLICE_COLUMN)

In [ ]:
##### Run this cell one time only
import pandas as pd
from ratio_split import ratio_splitting

VAL_SIZE = 0.15
CHROM_COLUMN = 'CHROM'
SPLICE_COLUMN = 'Splicing_types'
TRAIN_DATA_PATH = r'data_preparation\raw\train_val_data.csv' # Fill this only if interupted by error (no need to rerun the 1st cell)

try:
    train
except NameError:
    train = None
if train is None:
    train = pd.read_csv(TRAIN_DATA_PATH)

ratio_splitting(train, CHROM_COLUMN, SPLICE_COLUMN, VAL_SIZE)

In [ ]:
##### Run this cell one time only
from extract_embed import embed_train_val_folder

DATA_PATH = r"data_preparation\train_val"

embed_train_val_folder(root=DATA_PATH, replace=True) #Automately delete csv files after embedding -> change to False to keep csv files